In [9]:
# Biblioteca

import polars as pl
import numpy as np
import os
from scipy.spatial import KDTree

# Localização dos arquivos

pasta =  'tratados_10x10det'

# Retorna o nome do arquivo formatado
import re
def formatar_string(s):
   
    match = re.match(r'([a-zA-Z]+)([0-9]+(?:\.[0-9]+)?[Ee][0-9]+)', s)

    if match:
        
        palavra = match.group(1).capitalize()
        numero = match.group(2)

        return f'{palavra} {numero}'
    else:
        return s

# Posição dos tanques
numDetectors = 100
gridSize = 10
spacing = 10
area = 1.0

x_det = np.zeros(numDetectors)
y_det = np.zeros(numDetectors)
areas = np.full(numDetectors, area)

index = 0
halfGridSize = gridSize // 2

for i in range(gridSize):
    for j in range(gridSize):
        x_det[index] = (j - halfGridSize + 0.5) * spacing
        y_det[index] = (i - halfGridSize + 0.5) * spacing
        areas[index] = area
        index += 1

# Método KDTree para validação da triangulação

positions = np.column_stack((x_det, y_det))
tree = KDTree(positions)
d_max = 15

def is_valid_simulation(detections):
    detected_indices = np.where(detections)[0]
    if len(detected_indices) < 3:
        return False
    
    neighbors = []
    for tank in detected_indices:
        neighbors.append(set(tree.query_ball_point(positions[tank], d_max)))
    
    for i in range(len(detected_indices)):
        for j in range(i + 1, len(detected_indices)):
            if detected_indices[j] not in neighbors[i]:
                continue
            
            for k in range(j + 1, len(detected_indices)):
                if (
                    detected_indices[k] in neighbors[i]
                    and detected_indices[k] in neighbors[j]
                ):
                    return True
    return False

dataframes = []
for root, dirs, files in os.walk(pasta):
    for arquivo in files:
        if arquivo.endswith('_array'):
            caminho_completo = os.path.join(root, arquivo)  
            print(f'Processando arquivo: {arquivo}') 
            # Leitura do arquivo
            try:
                df = pl.read_csv(caminho_completo, has_header=False).filter(pl.col('column_1').str.contains('TRIG'))
                df = df.with_columns(pl.col("column_1").str.split(" ").alias("split_column"))
                df = df.with_columns(pl.col("split_column").list.get(0).alias("TRIG"))
                df = df.with_columns(pl.col("split_column").list.get(1).cast(pl.Int64).alias("positrons"))
                df = df.with_columns(pl.col("split_column").list.get(2).cast(pl.Int64).alias("electrons"))
                df = df.with_columns(pl.col("split_column").list.get(3).cast(pl.Int64).alias("muons_plus"))
                df = df.with_columns(pl.col("split_column").list.get(4).cast(pl.Int64).alias("muons_minus"))
                df = df.with_columns(pl.col("split_column").list.get(5).cast(pl.Float64).alias("time"))
                df = df.with_columns(
                    (pl.col("positrons") + pl.col("electrons") + pl.col("muons_plus") + pl.col("muons_minus")).alias("total_particles")
                )
                
                angles = pl.read_csv(caminho_completo, has_header=False).filter(pl.col('column_1').str.contains('EVTH'))
                angles = angles.with_columns(pl.col("column_1").str.split(" ").alias("split_column"))
                angles = angles.with_columns(
                    pl.col("split_column").list.get(5).cast(pl.Float64).alias("theta"),
                    pl.col("split_column").list.get(6).cast(pl.Float64).alias("phi")
                ).select(["theta", "phi"])

                # Adiciona uma coluna para identificar a simulação
                df = df.with_columns((pl.arange(0, df.height, eager=True) // 100).alias("simulation_id"))

                # Etapa de validação com KDTree
                valid_simulations = []
                for sim_id in df["simulation_id"].unique():
                    sim_data = df.filter(pl.col("simulation_id") == sim_id)
                    
                    detections = np.zeros(100, dtype=bool)
                    for row in sim_data.iter_rows(named=True):
                        trig_number = int(row["TRIG"][4:])  # Extrai o número do TRIG 
                        detections[trig_number - 1] = row["total_particles"] > 0  
                    
                    if is_valid_simulation(detections):
                        valid_simulations.append(sim_id)
                
                # Filtra apenas simulações válidas
                df = df.filter(pl.col("simulation_id").is_in(valid_simulations))

                # Nome formatado do arquivo
                name = formatar_string(arquivo)

                valid_simulations_sorted = sorted(valid_simulations)
                angle_df = pl.DataFrame({
                    "simulation_id": valid_simulations_sorted,
                    "theta": angles["theta"][:len(valid_simulations_sorted)].to_list(),
                    "phi": angles["phi"][:len(valid_simulations_sorted)].to_list()
                })
                
                # Fazer join com o DataFrame principal
                df = df.join(angle_df, on="simulation_id", how="left")

                # Transformação das contagens de partículas em listas e na média de contagens
                #df = df.group_by('TRIG').agg(pl.col("total_particles")).sort(pl.col("TRIG").str.extract(r"TRIG([0-9]*)", 1).cast(int))
                #df = df.with_columns(particles=pl.col('total_particles').list.mean())
                #df = df.with_columns(
                #    pl.lit(name.split()[0]).alias('composition'),
                #    pl.lit(name.split()[1]).alias('energy')
                #)
                #df = df.drop('total_particles')

                df = df.select(["TRIG", "total_particles", "simulation_id", "time", "theta", "phi"]).sort(  
                    pl.col("TRIG").str.extract(r"TRIG([0-9]*)", 1).cast(int)
                )
                df = df.rename({"total_particles": "particles"})

                # Adiciona composição e energia a cada linha
                df = df.with_columns(
                    pl.lit(name.split()[0]).alias("composition"),
                    pl.lit(name.split()[1]).alias("energy"),
                )                
 
                # Adiciona o dataframe à lista
                dataframes.append(df)
            except Exception as e:
                print(f'Erro ao processar o arquivo {caminho_completo}: {e}') 


# Verifica se a lista de dataframes não está vazia
if dataframes:
    # Combina todos os dataframes em um único dataframe
    df_final = pl.concat(dataframes)
    # Embaralha o dataframe
    df_final = df_final.sample(fraction = 1.0, shuffle = True)
    # Exibe o dataframe final
    print(df_final)
else:
    print("Nenhum dataframe foi criado. Verifique se há arquivos válidos na pasta e se os arquivos estão no formato correto.")

Processando arquivo: carbon1E15_array
Processando arquivo: carbon1E16_array
Processando arquivo: carbon1E17_array
Processando arquivo: carbon3.16E15_array
Processando arquivo: carbon3.16E16_array
Processando arquivo: iron1E15_array
Processando arquivo: iron1E16_array
Processando arquivo: iron1E17_array
Processando arquivo: iron3.16E15_array
Processando arquivo: iron3.16E16_array
Processando arquivo: nitrogen1E15_array
Processando arquivo: nitrogen1E16_array
Processando arquivo: nitrogen1E17_array
Processando arquivo: nitrogen3.16E15_array
Processando arquivo: nitrogen3.16E16_array
Processando arquivo: oxygen1E15_array
Processando arquivo: oxygen1E16_array
Processando arquivo: oxygen1E17_array
Processando arquivo: oxygen3.16E15_array
Processando arquivo: oxygen3.16E16_array
Processando arquivo: photon1E15_array
Processando arquivo: photon1E16_array
Processando arquivo: photon1E17_array
Processando arquivo: photon3.16E15_array
Processando arquivo: photon3.16E16_array
Processando arquivo:

In [11]:
# Normalização da densidade
density_min = df_final.select(pl.col('particles').min()).to_numpy()[0, 0]
density_max = df_final.select(pl.col('particles').max()).to_numpy()[0, 0]
    
df_normalized = df_final.with_columns(
      ((pl.col('particles') - density_min) / (density_max - density_min)).alias('particles_normalized')
    )
df_normalized
df_normalized.write_csv('data_neural_network_10x10.csv')

In [3]:
print(density_max,density_min)

61831 0


In [ ]:
df_final.filter(
    (pl.col("composition") == 'Proton') & (pl.col("energy") == '1E15') & (pl.col("TRIG") == 'TRIG4') & (pl.col("simulation_id") == 1)
)

TRIG,particles,simulation_id,time,theta,phi,composition,energy
str,i64,i64,f64,f64,f64,str,str
"""TRIG4""",0,1,0.0,31.174,-68.531,"""Proton""","""1E15"""


: 